In [1]:
import json, csv, copy

In [2]:
MAX_RULES = 4
ROWS_TO_PROCESS = 40000

In [3]:
#extract regulations from a row
def get_regulations(row,number=None):
    suffix = "_"+str(number) if number else ""
     
    regulation = {}
   
    rule = {}
    
    if len(row.get('zone'+suffix)):
        rule['activity'] = row.get('zone'+suffix)
    
    if row.get('reason'+suffix):
        rule['reason'] = row.get('reason'+suffix)
    
    if row.get('timeLimit'+suffix):
        rule['maxStay'] = int(row.get('time_limit'+suffix))
        
    if row.get('payment'+suffix):
        rule['payment'] = bool(row.get('payment'+suffix))
    
    if len(rule):
        regulation['rule'] = rule


            
    userClass = {}
    
    if row.get('classes'+suffix):
        userClass['classes'] = [row.get('classes'+suffix)]
    
    if len(userClass):
        regulation['userClass'] = userClass
        
    timeSpans = {}
     
    #Start with the timing:
    if row.get('days_of_week.days'+suffix):
        timeSpans['daysOfWeek'] = {
            'days' : [x.strip() for x in row.get('days_of_week.days'+suffix).split(",")]
        }
        
    if row.get('time_of_day.from'+suffix):
        timeSpans['timesOfDay'] = [{
            'from':row.get('time_of_day.from'+suffix),
            'until'  :row.get('time_of_day.to'+suffix) #Could add defaults here if necessary
        }]
    
    if len(timeSpans):
        regulation['timeSpans'] = [timeSpans]
    else:
        regulation['timeSpans'] = []
        
    # Now check if there are other days or other times of day:
    for suffix2 in ['b','c','d','e','f','g','h','i']:
        new_timeSpans = {}
            
        if row.get('days_of_week.days'+suffix+suffix2):
            new_timeSpans['daysOfWeek'] = {
                'days': [x.strip() for x in row.get('days_of_week.days'+suffix+suffix2).split(",")]
            }


        if row.get('time_of_day.from'+suffix+suffix2):
            if 'daysOfWeek' in new_timeSpans and regulation['timeSpans'][len(regulation["timeSpans"])-1]['daysOfWeek'] == new_timeSpans['daysOfWeek']:
                regulation['timeSpans'][len(regulation["timeSpans"])-1]['timesOfDay'].append({
                    'from': row.get('time_of_day.from' + suffix + suffix2),
                    'until': row.get('time_of_day.to' + suffix + suffix2) # Could add defaults here if necessary
                })
                
                del new_timeSpans['daysOfWeek']
            else:
                new_timeSpans['timesOfDay'] = [{
                    'from':row.get('time_of_day.from'+suffix+suffix2),
                    'until'  :row.get('time_of_day.to'+suffix+suffix2) #Could add defaults here if necessary
                }]

        if new_timeSpans:
            regulation['timeSpans'].append(new_timeSpans)

    for ts in regulation['timeSpans']:
        if 'daysOfWeek' in ts:
            if len(ts['daysOfWeek']['days']) == 7:
                del ts['daysOfWeek']
            
    payment = {}
    
    if row.get('payment_min' + suffix):
        payment = {
            'rates': [
                {
                    'fees': list(map(float, filter(lambda x: x is not "", [row.get('payment_min' + suffix), row.get('payment_max' + suffix)]))),
                    'durations': list(map(int, filter(lambda x: x is not "", [row.get('payment_min_interval' + suffix), row.get('payment_max_interval' + suffix)])))
                }
            ]
        }
        
        if len(payment['rates'][0]['fees']) == 0:
            del payment['rates'][0]['fees']

        if len(payment['rates'][0]['durations']) == 0:
            del payment['rates'][0]['durations']

    if row.get('method'+suffix):
        payment['method'] = row.get('method'+suffix)
        
    if row.get('payment_form'+suffix):
        payment['paymentForm'] = row.get('payment_form'+suffix)
        
    if len(payment):
        regulation['payment'] = payment
        
    if row.get('priority'+suffix):
        regulation['priority'] = int(row.get('priority'+suffix))
    
    return [regulation]

In [4]:
#Runs everything
import csv
import json

output_rows = []
with open('prepped_data.csv','r') as csv_file:
    csv_reader = csv.DictReader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        if row['location_start'].strip():
            line_count +=1;
            if line_count <= ROWS_TO_PROCESS:
                try:
                    #object level
                    obj = {
                        'location': {
                            'coordinates': json.loads(row['coordinates']),
                            'shstRefId':row['ShStRefID'],
                            'shstLocationSt':float(row['location_start']),
                            'shstLocationEnd':float(row['location_end']),
                            'sideOfStreet':row['sideOfStreet'],
                            'objectId': row['SpaceID'],
                            'derivedFrom': row['derived_from'].split(","),
                            'marker': row['marker'],
                            'streetName': row['streetName']
                        }
                    }

                    obj['regulations'] = get_regulations(row,"")

                    for i in range(2, MAX_RULES + 1):
                        if row.get("zone_"+str(i)):
                            obj['regulations'].extend(get_regulations(row,i))

                    if len(obj['regulations']):
                        output_rows.append(copy.deepcopy(obj))
                except ValueError as e:
                    print(e)
                    print(row)
                    # raise


with open('output.json','w') as outFile:
    json.dump(output_rows,outFile,indent=2)
    


In [5]:
output_rows_geojson = []

for item in output_rows:
    coordinates = item['location'].pop('coordinates')
    output_rows_geojson.append({
        'type':"Feature",
        'geometry':{
            'type':"LineString",
            'coordinates': coordinates
        },
        'properties': item
        
    })


In [6]:
with open('output.geojson','w') as outFile:
    json.dump({'type':'FeatureCollection','features':output_rows_geojson},outFile,indent=2)